------

In [9]:
%%time

from sentence_transformers import SentenceTransformer, models
from torch import nn

word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=256, do_lower_case=True)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), \
                                pooling_mode_mean_tokens=True, \
                                pooling_mode_cls_token=False,
                                pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

#print(model)
#print(model.get_sentence_embedding_dimension())

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CPU times: user 2 s, sys: 310 ms, total: 2.31 s
Wall time: 5.35 s


In [10]:
print("Max sequence length:", model.max_seq_length)

Max sequence length: 256


In [11]:
import datasets
sts = datasets.load_dataset('glue', 'stsb', split='validation')
sts = sts.map(lambda x: {'label': x['label'] / 5.0})
sts

Found cached dataset glue (/Users/olang/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /Users/olang/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-18d2b6495b7b8793.arrow


Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 1500
})

In [12]:
from sentence_transformers import InputExample
from tqdm import tqdm

samples = []
for sample in tqdm(sts):
    samples.append(InputExample(
        texts=[sample['sentence1'], sample['sentence2']],
        label=sample['label']
    )
)

100%|██████████| 1500/1500 [00:00<00:00, 14256.93it/s]


In [13]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    samples, write_csv=False
)

In [14]:
print(evaluator(model))

0.6146254070526513
